In [19]:
from src.utility import get_root
import pandas as pd
import os
from typing import Tuple, List, Optional

In [20]:
def estimate_flops(model: str, input_size: Tuple[int, int], training_strategy: str, sample_count: int, estimated_epochs: int) -> float:
    path_to_flops = os.path.join(get_root(), "data", "model_flops", "model_flops.xlsx")
    flops_df = pd.read_excel(path_to_flops)

    model_info = flops_df[flops_df["Model"] == model]

    if model_info.empty:
        raise ValueError(f"Model {model} not found in the flops database")
    
    model_type = model_info['Type'].iloc[0]
    original_input_size = model_info['Input Size'].iloc[0].split()[0]

    if model_type == 'Vision':
        width, height = map(int, original_input_size.split('x'))
        scaling = (input_size[0] * input_size[1]) / (width * height)
    else:
        scaling = input_size[0] / int(original_input_size)

    if training_strategy in ["Fine-tuning the whole model", "Full Training"]:
        return estimated_epochs * sample_count * model_info['FLOPs'].iloc[0] * scaling * 3
    elif training_strategy == "Last Layer Learning":
        return estimated_epochs * sample_count * 2 * model_info['Last Layer FLOPs'].iloc[0] + model_info['FLOPs'].iloc[0] * scaling
    else:
        raise ValueError(f"Unsupported training strategy: {training_strategy}")


In [28]:
def estimate_time(flops: float, gpu: str, training_strategy: str, tflops: str) -> float:
    path_to_gpu = os.path.join(get_root(), "data", "gpus.csv")
    gpu_df = pd.read_csv(path_to_gpu)
    gpu_info = gpu_df[gpu_df["name"] == gpu]

    if gpu_info.empty:
        raise ValueError(f"GPU {gpu} not found in the flops database")
    
    return flops / gpu_info[tflops].iloc[0] / 1e+12 if training_strategy in ["Full Training", "Fine-tuning the whole model"] else flops / gpu_info[tflops].iloc[0] / 1e+12 / 3

In [29]:
a = estimate_flops("fasterrcnn_resnet50_fpn", (256, 256), "Fine-tuning the whole model", 1000000, 40)
print(a)

2.106196446814041e+19


In [27]:
estimate_time(a, "RTX 3080 TI", "Fine-tuning the whole model")

617652.9169542642